In [17]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(separator="\n", chunk_size=600, chunk_overlap=100)

loader = UnstructuredFileLoader("./examples/chapter_one.md")

len(loader.load_and_split(text_splitter=splitter))

247